In [1]:
# basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xarray as xr
import time
from copy import deepcopy
import sys

# enable autoreload
%load_ext autoreload
%autoreload 2
# Variables from config file
sys.path.append('/home/yusukemh/github/yusukemh/StatisticalDownscaling/codes/')
from config import BASE_DIR, FILE_NAMES, LABELS, ATTRIBUTES, BEST_MODEL_COLUMNS, ISLAND_RANGES
from math import pi as PI

In [2]:
# load datasets
df_train = pd.read_csv(f"{BASE_DIR}/train.csv")
df_valid = pd.read_csv(f"{BASE_DIR}/valid.csv")
df_test = pd.read_csv(f"{BASE_DIR}/test.csv")
df_combined = pd.concat([df_train, df_valid, df_test])

In [3]:
df_combined.shape

(865561, 159)

In [4]:
df_original = pd.read_excel(f"{BASE_DIR}/FilledDataset2012.xlsx", sheet_name="Source")
X = []
for index, row in df_original.iterrows():
    if row.Year < 1948:
        # No need to keep data older than 1948 becase no data exists in netCDF files
        continue
    for i, cell in enumerate(row[2:]):
        X.append([row.SKN, row.Year, i + 1, cell])
df_nonan = pd.DataFrame(X, columns=['skn', 'year', 'month', 'method']).dropna()

In [8]:
valid_method = [
    'State/NCDC', 'State', 'NCDC', 'RAWS', 'Hydronet', 'SCAN', 'USGS',
    'Hydronet/NCDC', 'HaleNet', 'HC&S', 'AlanMair', 'AlanMair/State',
    'USGS/State', 'USGS/NCDC'
]
df_valid_method = pd.DataFrame({"method": valid_method})
df_valid_method = df_nonan.merge(right=df_valid_method, left_on='method', right_on='method')

# inner join with valid method
df_data_valid = df_combined.merge(right=df_valid_method, left_on=["skn", "year", "month"], right_on=['skn', 'year', 'month'])

df_data_valid.to_csv(f"{BASE_DIR}/nonfilled_dataset.csv", index=False)